### Install libraries required

This notebook will download the US County level Covid-19 case data from JHU Github and uses [Plotly](https://plotly.com/python/county-choropleth/) python package to generate GA county animation map from March 1st to mid August. As the first case of Georgia starts March 2020.
The python notebook requires several packages: Plotly, geopandas and finally ImageMagic to convert the series png files to GIF.
If you want your state animation, just change scope to "State name" and the start date.

In [1]:
#!pip install plotly
#!pip install plotly-geo
#!pip install geopandas==0.3.0
#!pip install pyshp==1.2.10
#!pip install shapely==1.6.3

import os
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from os.path import join, exists
from collections import OrderedDict
import requests
import plotly
import plotly.figure_factory as ff

data_dir = r'c:\tempdata'
timeseries_dir = r'c:\tempdata\infections_timeseries'

if not os.path.exists(data_dir):
    os.makedirs(data_dir)
if not exists(timeseries_dir):
    os.mkdir(timeseries_dir)

### Download Covid19 cases from JHU github

In [3]:
csvurl = 'https://raw.githubusercontent.com/JieYingWu/COVID-19_US_County-level_Summaries/master/data/infections_timeseries.csv'
resp = requests.get(csvurl)
if resp.status_code == 200:
    print('csv file exists')
else:
    print('csv file not found in jhu Web') 
with open(os.path.join(data_dir, "infections_timeseries.csv"), 'wb') as f:
    f.write(resp.content)

csv file exists


In [11]:
timeseries = pd.read_csv(join(data_dir, 'infections_timeseries.csv'), dtype=str)
# labels = timeseries.keys()[1:]
labels = timeseries.keys()[40:] # start March 1st as Georgia first case. you need to change this to different date.
per_what = 0 # 10000

# Georgia case starts March 1, 2020
# JHU Data infections start date is Jan 22, 2020
start = dt.date(2020, 3, 1)

In [12]:
print (labels[1])

3/1/20


In [13]:
datas = [OrderedDict() for _ in labels]
for i in range(len(datas)):
    for fips, st, value in zip(timeseries['FIPS'],timeseries['Combined_Key'], timeseries[labels[i]]):
        if 'Georgia' in st:
            # Get the number of visits per per_what people.
            if bool(per_what):
                datas[i][fips] = float(value) / float(pops[fips]) * per_what
            else:
                datas[i][fips] = float(value)

In [16]:
# Define the bounds of the bins based on quantiles of every data point.
values = np.array(sum([list(data.values()) for data in datas], []))
print(np.max(values), len(datas))
binning_endpoints = [1, 5, 10, 50, 100, 500, 1000, 5000, 10000, 15000, 20000, 25000]
# max_val = np.max(values)
# step_size = 5000 # round(max_val / 15, -3)
# binning_endpoints = list(np.arange(0, max_val, step_size))
# binning_endpoints = list(np.arange(0, 10.1, 0.5))
# binning_endpoints = list(np.quantile(values, [0.5, 0.9, 0.95, 0.99, 0.999]))

# Get a color map.
cmap = plt.get_cmap('Reds')

# Define the colors based on these bins using a colormap from matplotlib.
num_points = len(binning_endpoints) + 1
colors = [(0.9, 0.9, 0.9)] + [cmap((i+1) / num_points) for i in range(num_points)]
colorscale = [f'rgb({t[0]}, {t[1]}, {t[2]})' for t in colors]

20276.0 163


In [17]:
for ii, data in enumerate(datas):
    # Get the ordinal 
    date = dt.date.fromordinal(start.toordinal() + ii)
    datestring = date.strftime('%B ') + str(int(date.day)) + f' {date.year}'
    fig = ff.create_choropleth(
        fips=list(data.keys()),     # List of FIPS codes.
        values=list(data.values()), # 
        scope=['Georgia'],
        binning_endpoints=binning_endpoints,
        county_outline={'color': 'rgb(255,255,255)', 'width': 0.05},
        colorscale=colorscale,
#         round_legend_values=True,
#         title=f'{datestring.rjust(15)} Grocery Visits per {per_what}'
        title=f'{datestring.rjust(15)} Georgia COVID-19 Cases'
    )
    fig.layout.template = None
    # Order the files by date.
    filename = join(timeseries_dir, f'{date.isoformat()}.png')
    
    # write the image or show it.
    fig.write_image(filename)
    if ii == 0:
        fig.show()
    print(f'Wrote image {ii + 1}/{len(datas)}')

Wrote image 1/163
Wrote image 2/163
Wrote image 3/163
Wrote image 4/163
Wrote image 5/163
Wrote image 6/163
Wrote image 7/163
Wrote image 8/163
Wrote image 9/163
Wrote image 10/163
Wrote image 11/163
Wrote image 12/163
Wrote image 13/163
Wrote image 14/163
Wrote image 15/163
Wrote image 16/163
Wrote image 17/163
Wrote image 18/163
Wrote image 19/163
Wrote image 20/163
Wrote image 21/163
Wrote image 22/163
Wrote image 23/163
Wrote image 24/163
Wrote image 25/163
Wrote image 26/163
Wrote image 27/163
Wrote image 28/163
Wrote image 29/163
Wrote image 30/163
Wrote image 31/163
Wrote image 32/163
Wrote image 33/163
Wrote image 34/163
Wrote image 35/163
Wrote image 36/163
Wrote image 37/163
Wrote image 38/163
Wrote image 39/163
Wrote image 40/163
Wrote image 41/163
Wrote image 42/163
Wrote image 43/163
Wrote image 44/163
Wrote image 45/163
Wrote image 46/163
Wrote image 47/163
Wrote image 48/163
Wrote image 49/163
Wrote image 50/163
Wrote image 51/163
Wrote image 52/163
Wrote image 53/163
Wr

### This need ImageMagic to convert png to gif

In [27]:
%cd c:\tempdata\infections_timeseries
!convert -delay 50 -loop 0 *.png gacases.gif

c:\tempdata\infections_timeseries


### Citation paper
@article{killeenCountylevelDatasetInforming2020,
  title = {A {{County}}-Level {{Dataset}} for {{Informing}} the {{United States}}' {{Response}} to {{COVID}}-19},
  author = {Killeen, Benjamin D. and Wu, Jie Ying and Shah, Kinjal and Zapaishchykova, Anna and Nikutta, Philipp and Tamhane, Aniruddha and Chakraborty, Shreya and Wei, Jinchi and Gao, Tiger and Thies, Mareike and Unberath, Mathias},
  year = {2020},
  month = apr,
  archivePrefix = {arXiv},
  eprint = {1909.11730},
  eprinttype = {arxiv}
}